In [90]:
#importing basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [91]:
# data = pd.read_csv('Training Dataset.arff', header=None, comment='@')
data = pd.read_csv("urldata.csv")
data = data.drop(['Domain'], axis = 1).copy()

In [92]:
data.isnull().sum()



Have_IP          0
Have_At          0
URL_Length       0
URL_Depth        0
Redirection      0
https_Domain     0
TinyURL          0
Prefix/Suffix    0
DNS_Record       0
Web_Traffic      0
Domain_Age       0
Domain_End       0
iFrame           0
Mouse_Over       0
Right_Click      0
Web_Forwards     0
Google_Index     0
count_dot        0
count_www        0
count_per        0
count_ques       0
count_hyphen     0
count_equal      0
Label            0
dtype: int64

In [93]:
# shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,...,Right_Click,Web_Forwards,Google_Index,count_dot,count_www,count_per,count_ques,count_hyphen,count_equal,Label
0,0,0,0,0,0,0,0,0,0,1,...,1,1,1,2,0,0,0,0,0,1
1,0,0,1,3,0,0,0,0,0,1,...,1,0,1,3,0,5,1,0,1,0
2,0,0,1,10,0,0,0,0,0,1,...,1,0,1,1,0,0,0,0,0,0
3,0,0,1,2,0,0,0,0,0,1,...,1,1,1,2,0,0,0,10,0,0
4,0,0,1,3,0,0,0,0,0,0,...,1,0,1,2,0,0,0,0,0,0


In [94]:
# Sepratating & assigning features and target columns to X & y
y = data['Label']
X = data.drop('Label',axis=1)
# X = data.iloc[:, :-1]
# y = data.iloc[:, -1]

In [95]:
X.shape, y.shape

column_name=list(X.columns)

In [96]:
# Splitting the dataset into train and test sets: 80-20 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, random_state = 12)
X_train.shape, X_test.shape

((8000, 23), (2000, 23))

In [97]:
import pickle

dt = pickle.load(open('dt.pkl', 'rb'))
logi = pickle.load(open('logi.pkl', 'rb'))
rf = pickle.load(open('rf.pkl', 'rb'))
xg = pickle.load(open('xg.pkl', 'rb'))
svm = pickle.load(open('svm.pkl', 'rb'))

In [98]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

clf = [('dt',dt),('xg',xg),('rf',rf),('svm',svm)] #list of (str, estimator)
stack_model = StackingClassifier( estimators = clf,final_estimator = logi)

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=0, shuffle=True)
results = cross_val_score(stack_model, X_train, y_train, cv=kfold)
print("K-Fold Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))


K-Fold Accuracy: 98.375% (0.454%)


In [99]:
from sklearn.metrics import accuracy_score
stack_model.fit(X_train,y_train)


StackingClassifier(estimators=[('dt',
                                DecisionTreeClassifier(max_depth=20,
                                                       max_features='auto',
                                                       random_state=0)),
                               ('xg',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=Non...
                                              monotone_constraints=None,
                                              n_estimators=200, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=0, ...)),
                               ('rf',
                                RandomForestClassifier(max_depth=20,
                                                       max_features='auto',
                                                       random_state=0)),
                               ('svm',
                                SVC(C=2, gamma=0.5, probability=True,
                                    random_state=0))],
                   final_estimator=LogisticRegression(C=1,
                                                      class_weight='balanced',
                                                      penalty='l1',
                                                      random_state=0,
                                                      solver='liblinear'))

In [103]:
y_pred=stack_model.predict(X_test)
acc_test= accuracy_score(y_test,y_pred)
print(acc_test*100)

98.85


In [101]:
from sklearn.metrics import classification_report


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1026
           1       0.98      0.99      0.99       974

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [102]:
import pickle

with open('ensemble.pkl', 'wb') as file:
    pickle.dump(stack_model, file)